In [ ]:
!nvidia-smi

!pip install -q tensorflow scikit-learn

import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB4


Sun Nov 16 23:54:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             52W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Adjust paths if your folder names differ
drive_train_dir = "/content/drive/MyDrive/ASL_data/training"
drive_test_dir  = "/content/drive/MyDrive/ASL_data/test"

# Copy to fast local storage
!cp -r "$drive_train_dir" /content/
!cp -r "$drive_test_dir" /content/

print("Local training subfolders:")
!ls /content/training | head

print("\nLocal test files:")
!ls /content/test | head


Local training subfolders:
A
B
C
D
del
E
F
G
H
I

Local test files:
A_test.jpg
B_test.jpg
C_test.jpg
D_test.jpg
E_test.jpg
F_test.jpg
G_test.jpg
H_test.jpg
I_test.jpg
J_test.jpg


In [ ]:
!ls /content/training


A  C  del  F  H  J  L  N	O  Q  S      T	V  X  Z
B  D  E    G  I  K  M  nothing	P  R  space  U	W  Y


In [ ]:
IMG_SIZE   = (224, 224)   # you can drop to (160,160) if needed
BATCH_SIZE = 64           # A100 can handle 64–128 easily

train_dir = "/content/training"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)
print("Num classes:", num_classes)


Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Num classes: 29


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)


In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1, 0.1),
])


In [ ]:
base_model = EfficientNetB4(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SIZE + (3,)
)

base_model.trainable = False  # Stage 1: freeze backbone


In [ ]:
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 7, 7, 1792)     │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 29)             │        51,997 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,725,820 (67.62 MB)

 Trainable params: 51,997 (203.11 KB)

 Non-trainable params: 17,673,823 (67.42 MB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

EPOCHS_STAGE1 = 5  # warm-up

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_STAGE1
)


Epoch 1/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 154s 86ms/step - accuracy: 0.5346 - loss: 1.8148 - val_accuracy: 0.8836 - val_loss: 0.4822
Epoch 2/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 98s 81ms/step - accuracy: 0.7968 - loss: 0.7467 - val_accuracy: 0.9134 - val_loss: 0.3260
Epoch 3/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 98s 82ms/step - accuracy: 0.8278 - loss: 0.5973 - val_accuracy: 0.9211 - val_loss: 0.2795
Epoch 4/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 98s 82ms/step - accuracy: 0.8465 - loss: 0.5235 - val_accuracy: 0.9335 - val_loss: 0.2357
Epoch 5/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 98s 82ms/step - accuracy: 0.8542 - loss: 0.4873 - val_accuracy: 0.9445 - val_loss: 0.2037


In [ ]:
# Unfreeze the backbone for fine-tuning
base_model.trainable = True

# Optionally freeze the lower layers and train only the top ~150 layers
for layer in base_model.layers[:-150]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

EPOCHS_STAGE2 = 5

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_STAGE2
)


Epoch 1/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 190s 130ms/step - accuracy: 0.7376 - loss: 0.9126 - val_accuracy: 0.9801 - val_loss: 0.0630
Epoch 2/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 145s 125ms/step - accuracy: 0.9566 - loss: 0.1375 - val_accuracy: 0.9947 - val_loss: 0.0198
Epoch 3/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 145s 125ms/step - accuracy: 0.9790 - loss: 0.0658 - val_accuracy: 0.9973 - val_loss: 0.0093
Epoch 4/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 145s 125ms/step - accuracy: 0.9880 - loss: 0.0375 - val_accuracy: 0.9992 - val_loss: 0.0042
Epoch 5/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 145s 125ms/step - accuracy: 0.9919 - loss: 0.0255 - val_accuracy: 0.9995 - val_loss: 0.0024


In [ ]:
val_images = []
val_labels = []

for batch_imgs, batch_lbls in val_ds:
    val_images.append(batch_imgs.numpy())
    val_labels.append(batch_lbls.numpy())

val_images = np.vstack(val_images)
val_labels = np.vstack(val_labels)

print("Val images shape:", val_images.shape)
print("Val labels shape:", val_labels.shape)


Val images shape: (17400, 224, 224, 3)
Val labels shape: (17400, 29)


In [ ]:
pred_probs = model.predict(val_images, batch_size=BATCH_SIZE)
y_pred = np.argmax(pred_probs, axis=1)
y_true = np.argmax(val_labels, axis=1)

print("Classification report:")
print(classification_report(y_true, y_pred, target_names=class_names))

macro_f1 = f1_score(y_true, y_pred, average="macro")
weighted_f1 = f1_score(y_true, y_pred, average="weighted")

print("Macro F1:", macro_f1)
print("Weighted F1:", weighted_f1)


272/272 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step
Classification report:
              precision    recall  f1-score   support

           A       1.00      1.00      1.00       623
           B       1.00      1.00      1.00       634
           C       1.00      1.00      1.00       601
           D       1.00      1.00      1.00       643
           E       1.00      0.99      1.00       548
           F       1.00      1.00      1.00       646
           G       1.00      1.00      1.00       612
           H       1.00      1.00      1.00       626
           I       1.00      1.00      1.00       607
           J       1.00      1.00      1.00       626
           K       1.00      1.00      1.00       553
           L       1.00      1.00      1.00       593
           M       1.00      1.00      1.00       594
           N       1.00      1.00      1.00       555
           O       1.00      1.00      1.00       595
           P       1.00      1.00      1.00       606
           Q   

In [ ]:
import cv2

test_dir = "/content/test"

test_files = sorted(os.listdir(test_dir))
print("Number of test images:", len(test_files))
print(test_files[:10])


Number of test images: 29
['A_test.jpg', 'B_test.jpg', 'C_test.jpg', 'D_test.jpg', 'E_test.jpg', 'F_test.jpg', 'G_test.jpg', 'H_test.jpg', 'I_test.jpg', 'J_test.jpg']


In [ ]:
for file in test_files:
    img_path = os.path.join(test_dir, file)
    img = cv2.imread(img_path)

    # SKIP files that failed to load
    if img is None:
        print(f"Could not read {file}, skipping.")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = preprocess_input(img.astype(np.float32))

    img_batch = np.expand_dims(img, axis=0)
    probs = model.predict(img_batch, verbose=0)

    pred_idx = np.argmax(probs)
    pred_class = class_names[pred_idx]
    confidence = probs[0, pred_idx]

    print(f"{file:25s} → {pred_class:10s} (conf: {confidence:.3f})")


A_test.jpg                → A          (conf: 0.999)
B_test.jpg                → B          (conf: 1.000)
C_test.jpg                → C          (conf: 1.000)
D_test.jpg                → D          (conf: 0.999)
E_test.jpg                → E          (conf: 1.000)
F_test.jpg                → F          (conf: 1.000)
G_test.jpg                → G          (conf: 1.000)
H_test.jpg                → H          (conf: 1.000)
I_test.jpg                → I          (conf: 0.999)
J_test.jpg                → J          (conf: 1.000)
K_test.jpg                → K          (conf: 1.000)
L_test.jpg                → L          (conf: 1.000)
M_test.jpg                → M          (conf: 1.000)
N_test.jpg                → N          (conf: 1.000)
O_test.jpg                → O          (conf: 1.000)
P_test.jpg                → P          (conf: 1.000)
Q_test.jpg                → Q          (conf: 1.000)
R_test.jpg                → R          (conf: 1.000)
S_test.jpg                → S          (conf: 

In [ ]:
model_path = "/content/efficientnetb4_asl.h5"
model.save(model_path)
print("Saved model to:", model_path)


Saved model to: /content/efficientnetb4_asl.h5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/efficientnetb4_asl.h5 /content/drive/MyDrive/efficientnetb4_asl.h5
print("Model copied to Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model copied to Drive.
